Loading Data

Integrating Smart meter measurement data in network

In [ ]:
import Data_preprocessing_BH
from Data_preprocessing_BH import *
import os
import importlib
import random as rnd
importlib.reload(Data_preprocessing_BH)
import create_BH_net
from create_BH_net import *
importlib.reload(create_BH_net)
import Power_flow
from Power_flow import *

# Load the data
path = r"C:\Users\lukas\OneDrive - Danmarks Tekniske Universitet\SYNERGIES Project -- DTU Team - Lukas Tej Marner_SA Shared Folder\digitalwin1-main\digitalwin1-main"

file = 'demo_smart_meter_measurements.csv'
new_path = os.path.join(path, file)

demo_smart_meter_measurements = pd.read_csv(new_path, sep=';')
print(demo_smart_meter_measurements.head())
print(demo_smart_meter_measurements.columns)



# Create a prod_or_con DataFrame that matches the structure
prod_or_con = pd.DataFrame(index=[0], columns=demo_smart_meter_measurements.columns)

# Mark columns based on whether they are production or consumption
for col in demo_smart_meter_measurements.columns:
    if 'Production' in col:
        prod_or_con.loc[0, col] = -2  # Production marker
    elif 'Consumption' in col:
        prod_or_con.loc[0, col] = -1  # Consumption marker
    else:
        prod_or_con.loc[0, col] = None  # Other columns



measurement_number = rnd.randint(3, len(demo_smart_meter_measurements))
#smart_meter_measurements_sorted = fix_datetime(demo_smart_meter_measurements)
#measurement_number = 6473
# measurement_number = 1
#print('Index of measurement:', (measurement_number+3))
#print('Datetime:', (smart_meter_measurements_sorted['datetime'][measurement_number]))
#measurement_prod_cons = aggregated_measurements_substation(smart_meter_measurements_sorted, measurement_number = measurement_number)
measurement_prod_cons = aggregated_measurements_substation_demo(demo_smart_meter_measurements, prod_or_con,measurement_number = measurement_number)


file = 'Bornholm 20220706.xlsx'
new_path = os.path.join(path, file)

net = pp.create_empty_network()

net = load_data(new_path, net)



net = net_60kV_measurements(net, measurement_prod_cons)

# Print basic components of the network
print("Network components:")
for component in net.keys():
    if isinstance(net[component], pd.DataFrame) and not net[component].empty:
        print(f"- {component}: {len(net[component])} elements")
import pandapower.plotting as plot
import matplotlib.pyplot as plt

# Simple plotting
plot.simple_plot(net)
plt.show()

# If you have geographic data:
if any(net.bus.has_geo_data):
    plot.plotly.simple_plotly(net)

In [ ]:
#smart_meter_measurements.head()

In [ ]:
pp.runpp(net)

In [ ]:
# Path to CSV and new path for load_data function
measurement_prod_cons_csv_path = 'demo_smart_meter_measurements.csv'

# Perform power flow analysis
datetimes, secure_list, voltage_violations_list, overloaded_lines_list, overloaded_transformers_list, line_loadings_list, trafo_loadings_list, bus_voltages_injections_list = perform_power_flow_analysis(measurement_prod_cons_csv_path, new_path)

# Define the path to the Excel file
excel_path = 'results_powerflow_engine.xlsx'

# Write results to the Excel file
write_results_to_excel(excel_path, datetimes, secure_list, voltage_violations_list, overloaded_lines_list, overloaded_transformers_list, line_loadings_list, trafo_loadings_list, bus_voltages_injections_list)

# Open the Excel file
os.startfile(excel_path)

Integrating SCADA measurements in network

In [8]:


# Define timestamp
import Data_preprocessing_BH 
from Data_preprocessing_BH import *
import create_BH_net
from create_BH_net import *
importlib.reload(create_BH_net)
importlib.reload(Data_preprocessing_BH)
import os
timestamp = '2022-03-09 17:30:00'
timestamp = pd.to_datetime(timestamp)

new_path1 = r"C:\Users\lukas\Synergies_Local"
# List of substations with associated CSV and voltage
substations = [
    ('Allinge', 'lok_10kvskinnespend'),
    ('Aakirkeby', 't1s_spending'),
    ('Bodilsker', 'lok_10kvskinnespend'),
    ('Gudhjem', 't1s_10kvskinnespend'),
    ('Hasle', ''),
    ('Nexo', ''),
    ('Olsker', ''),
    ('Osterlars', ''),
    ('Povlsker', ''),
    ('RNO', 'fel_10kvskinnespend'),
    ('RNS', 'lok_10kvskinnespend'),
    ('Snorrebakken', 't1s_10kvskinnespend'),
    ('Svaneke', ''),
    ('Vesthavnen', 'fel_skinnespending'),
    ('Viadukten', 't1s_10kvskinnespend'),
]


net = pp.create_empty_network()


net = load_data(new_path1, net)

smart_meter_measurement_vaerket = measurement_prod_cons

df_SCADA_meas = make_SCADA_df(timestamp, substations)

meas_vaerket = pd.DataFrame({'Substation': ['Vaerket'], 'Active Power': [measurement_prod_cons['Consumption'].values[0]-measurement_prod_cons['Production'].values[0]], 'Reactive Power': [0]})

df_SCADA_meas = pd.concat([df_SCADA_meas, meas_vaerket], ignore_index=True)

net_meas = net_60kV_SCADA_measurements(net, df_SCADA_meas)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\lukas\\Synergies_Local'

In [ ]:
pp.runpp(net_meas, numba=False)

Perform Power flow

In [ ]:
import Power_flow
from Power_flow import *
secure, voltage_violations, overloaded_lines, overloaded_transformers, line_loadings, trafo_loadings, bus_voltages_injections = check_power_system_violations(net_meas)

In [ ]:
df_SCADA_meas.head()

In [ ]:
import pandas as pd
import Data_preprocessing_BH
importlib.reload(Data_preprocessing_BH)
# Initialize an empty list to store the dataframes
df_list = []

# Assume you have a list of timestamps (replace this with your actual timestamps)
timestamps = ['2022-03-09 17:30:00', '2022-03-09 17:45:00', '2022-03-09 18:00:00']

for timestamp in timestamps:
    # Generate the SCADA measurements for the given timestamp
    df_SCADA_meas = make_SCADA_df(timestamp, substations)
    timestamp_dt = pd.to_datetime(timestamp)
    matching_row = smart_meter_measurements_sorted[smart_meter_measurements_sorted['datetime'] == timestamp_dt]

    # Get the index of the matching row(s)
    measurement_number = matching_row.index[0]
    print(measurement_number)

    measurement_prod_cons = aggregated_measurements_substation_demo(smart_meter_measurements_sorted, prod_or_con,measurement_number = measurement_number)
    print('Datetime:', (smart_meter_measurements_sorted['datetime'][measurement_number]))


    meas_vaerket = pd.DataFrame({
        'Substation': ['Vaerket'], 
        'Active Power': [measurement_prod_cons['Consumption'].values[0]+measurement_prod_cons['Production'].values[0]], 
        'Reactive Power': [0]})

    # Concatenate 'Vaerket' measurements with the rest of the SCADA measurements
    df_SCADA_meas = pd.concat([df_SCADA_meas, meas_vaerket], ignore_index=True)

    # Add the current timestamp as a new column
    df_SCADA_meas['datetime'] = timestamp
    
    # Append the dataframe to the list
    df_list.append(df_SCADA_meas)

# Concatenate all dataframes into a single dataframe
final_df = pd.concat(df_list, ignore_index=True)

# Reorder the columns to have 'Datetime' first
final_df = final_df[['datetime', 'Substation', 'Active Power', 'Reactive Power']]

# Write the dataframe to a CSV file
final_df.to_csv('demo_scada_measurements.csv', index=False)

print("CSV file created successfully.")


In [ ]:
# Path to CSV and new path for load_data function
import Power_flow
import Data_preprocessing_BH
from Power_flow import *
importlib.reload(Power_flow)
importlib.reload(Data_preprocessing_BH)
measurement_prod_cons_csv_path = 'demo_scada_measurements.csv'

# Perform power flow analysis
datetimes, secure_list, voltage_violations_list, overloaded_lines_list, overloaded_transformers_list, line_loadings_list, trafo_loadings_list, bus_voltages_injections_list = perform_power_flow_analysis(measurement_prod_cons_csv_path, new_path, measurement_type = 'scada')

# Define the path to the Excel file
excel_path = 'results_powerflow_engine.xlsx'

# Write results to the Excel file
write_results_to_excel(excel_path, datetimes, secure_list, voltage_violations_list, overloaded_lines_list, overloaded_transformers_list, line_loadings_list, trafo_loadings_list, bus_voltages_injections_list)

# Open the Excel file
os.startfile(excel_path)

Perform contingency analysis

In [ ]:
df_SCADA_meas

In [ ]:
import pandas as pd
import Power_flow
import numpy as np
from Power_flow import *
from Contingency import*
import Contingency
import Data_preprocessing_BH
import pandapower as pp
import importlib
importlib.reload(Power_flow)
importlib.reload(Contingency)

n_scenarios = 10
scenarios = []
outages_list = []
voltage_violations_list = []
overloaded_lines_list = []
overloaded_transformers_list = []
line_loadings_list = []
trafo_loadings_list = []
bus_voltages_injections_list = []
scada_new_list = []



    
scenarios.append('Current_operating_point')
net = pp.create_empty_network()
net = load_data(new_path, net)
net_meas = net_60kV_SCADA_measurements(net, df_SCADA_meas)

# measurement_prod_cons_new = measurement_prod_cons.drop(columns = ['Production'])
outages, vvl, oll, otl, lll, tll, bvil = contingency_analysis_with_violations(net_meas)
outages_list.append(outages)
voltage_violations_list.append(vvl)
overloaded_lines_list.append(oll)
overloaded_transformers_list.append(otl)
line_loadings_list.append(lll)
trafo_loadings_list.append(tll)
bus_voltages_injections_list.append(bvil)
scada_new_list.append(df_SCADA_meas.copy())

scada_pf = df_SCADA_meas['Active Power'] / np.sqrt(df_SCADA_meas['Active Power']**2 + df_SCADA_meas['Reactive Power']**2)



for i in range(n_scenarios):
    print(i)
    
    scenarios.append(f'Scenario_{i}')

    net = pp.create_empty_network()
    net = load_data(new_path, net)
    new_active_power = sample_normal_distributions(df_SCADA_meas['Active Power'])
    scada_new = df_SCADA_meas
    scada_new['Active Power'] = new_active_power
    scada_new['Reactive Power'] = calculate_reactive_power(df_SCADA_meas['Active Power'], scada_pf)
    net_meas = net_60kV_SCADA_measurements(net, scada_new)
    outages, vvl, oll, otl, lll, tll, bvil = contingency_analysis_with_violations(net_meas)
    outages_list.append(outages)
    voltage_violations_list.append(vvl)
    overloaded_lines_list.append(oll)
    overloaded_transformers_list.append(otl)
    line_loadings_list.append(lll)
    trafo_loadings_list.append(tll)
    bus_voltages_injections_list.append(bvil)
    scada_new_list.append(scada_new.copy())

excel_path = 'results_contingency_analysis.xlsx'
save_results_to_excel(scenarios, outages_list, voltage_violations_list, overloaded_lines_list, overloaded_transformers_list, line_loadings_list, trafo_loadings_list, bus_voltages_injections_list, scada_new_list, excel_path)
# Open the Excel file
os.startfile(excel_path)

Flexibility engine

In [ ]:


# Define timestamp
import Data_preprocessing_BH 
from Data_preprocessing_BH import *
import create_BH_net
from create_BH_net import *

import os
timestamp = '2022-03-09 17:30:00'
timestamp = pd.to_datetime(timestamp)

new_path = r"C:\Users\lukas\OneDrive - Danmarks Tekniske Universitet\SYNERGIES Project -- DTU Team - Lukas Tej Marner_SA Shared Folder"
# List of substations with associated CSV and voltage
substations = [
    ('Allinge', 'lok_10kvskinnespend'),
    ('Aakirkeby', 't1s_spending'),
    ('Bodilsker', 'lok_10kvskinnespend'),
    ('Gudhjem', 't1s_10kvskinnespend'),
    ('Hasle', ''),
    ('Nexo', ''),
    ('Olsker', ''),
    ('Osterlars', ''),
    ('Povlsker', ''),
    ('RNO', 'fel_10kvskinnespend'),
    ('RNS', 'lok_10kvskinnespend'),
    ('Snorrebakken', 't1s_10kvskinnespend'),
    ('Svaneke', ''),
    ('Vesthavnen', 'fel_skinnespending'),
    ('Viadukten', 't1s_10kvskinnespend'),
]

net = pp.create_empty_network()

net = load_data(new_path, net)

smart_meter_measurement_vaerket = measurement_prod_cons

df_SCADA_meas = make_SCADA_df(timestamp, substations)

meas_vaerket = pd.DataFrame({'Substation': ['Vaerket'], 'Active Power': [measurement_prod_cons['Consumption'].values[0]-measurement_prod_cons['Production'].values[0]], 'Reactive Power': [0]})

df_SCADA_meas = pd.concat([df_SCADA_meas, meas_vaerket], ignore_index=True)

net_meas = net_60kV_SCADA_measurements(net, df_SCADA_meas)


In [ ]:
def fill_load(net):
    df = net.load

    # Step 1: Identify all buses (in this case 5 buses: 0, 1, 2, 3, 4)
    total_buses = len(net.bus)
    all_buses = pd.Series(range(total_buses))

    # Step 2: Merge the original DataFrame with a DataFrame containing all buses to identify missing ones
    df_all_buses = pd.DataFrame({'bus': all_buses})

    # Step 3: Perform an outer join to fill missing buses and set the NaN values to 0 for the necessary columns
    df_filled = df_all_buses.merge(df, on='bus', how='left')
    # # Step 4: Replace NaN values with 0 where necessary (or fill with a specific default value)
    df_filled['name'].fillna('Unknown', inplace=True)
    df_filled['p_mw'].fillna(0.0, inplace=True)
    df_filled['q_mvar'].fillna(0.0, inplace=True)
    df_filled['const_z_percent'].fillna(0.0, inplace=True)
    df_filled['const_i_percent'].fillna(0.0, inplace=True)
    df_filled['sn_mva'].fillna(0.0, inplace=True)
    df_filled['scaling'].fillna(1.0, inplace=True)
    df_filled['in_service'].fillna(True, inplace=True)
    df_filled['type'].fillna('Unknown', inplace=True)
    # df_filled['controllable'].fillna(False, inplace=True)
    # Step 5: Output the filled DataFrame
    net.load = df_filled
    return net.load

def fill_gen(net):
    df = net.res_gen
    df['bus'] = net.gen['bus']

    # Step 1: Identify all buses (in this case 5 buses: 0, 1, 2, 3, 4)
    total_buses = len(net.bus)
    all_buses = pd.Series(range(total_buses))
    # print(df_sgen)

    # df = pd.DataFrame(df)

    # df = df.merge(df_sgen, on ='bus', how ='left')
    # print(df)

    # Step 2: Merge the original DataFrame with a DataFrame containing all buses to identify missing ones
    df_all_buses = pd.DataFrame({'bus': all_buses})

    # Step 3: Perform an outer join to fill missing buses and set the NaN values to 0 for the necessary columns
    df_filled = df_all_buses.merge(df, on='bus', how='left')
    # # Step 4: Replace NaN values with 0 where necessary (or fill with a specific default value)
    df_filled['p_mw'].fillna(0.0, inplace=True)
    df_filled['q_mvar'].fillna(0.0, inplace=True)
    df_filled['va_degree'].fillna(0.0, inplace=True)
    df_filled['vm_pu'].fillna(1.0, inplace=True) 
    # Step 5: Output the filled DataFrame
    net.res_gen = df_filled
    return net.res_gen

def fill_sgen(net):
    df = net.res_sgen
    df['bus'] = net.sgen['bus']

    # Step 1: Identify all buses (in this case 5 buses: 0, 1, 2, 3, 4)
    total_buses = len(net.bus)
    all_buses = pd.Series(range(total_buses))
    # print(df_sgen)

    # df = pd.DataFrame(df)

    # df = df.merge(df_sgen, on ='bus', how ='left')
    # print(df)

    # Step 2: Merge the original DataFrame with a DataFrame containing all buses to identify missing ones
    df_all_buses = pd.DataFrame({'bus': all_buses})

    # Step 3: Perform an outer join to fill missing buses and set the NaN values to 0 for the necessary columns
    df_filled = df_all_buses.merge(df, on='bus', how='left')
    # # Step 4: Replace NaN values with 0 where necessary (or fill with a specific default value)
    df_filled['p_mw'].fillna(0.0, inplace=True)
    df_filled['q_mvar'].fillna(0.0, inplace=True)
    # Step 5: Output the filled DataFrame
    net.res_sgen = df_filled
    return net.res_sgen

def fill_ext_grid(net):
    df = net.res_ext_grid
    df['bus'] = net.ext_grid['bus']

    # Step 1: Identify all buses (in this case 5 buses: 0, 1, 2, 3, 4)
    total_buses = len(net.bus)
    all_buses = pd.Series(range(total_buses))
    # print(df_sgen)

    # df = pd.DataFrame(df)

    # df = df.merge(df_sgen, on ='bus', how ='left')
    # print(df)

    # Step 2: Merge the original DataFrame with a DataFrame containing all buses to identify missing ones
    df_all_buses = pd.DataFrame({'bus': all_buses})

    # Step 3: Perform an outer join to fill missing buses and set the NaN values to 0 for the necessary columns
    df_filled = df_all_buses.merge(df, on='bus', how='left')
    # # Step 4: Replace NaN values with 0 where necessary (or fill with a specific default value)
    df_filled['p_mw'].fillna(0.0, inplace=True)
    df_filled['q_mvar'].fillna(0.0, inplace=True)
    # Step 5: Output the filled DataFrame
    net.res_ext_grid = df_filled
    return net.res_ext_grid

def create_Y_bus(net):
    NB = len(net.bus)
    y = np.zeros((NB, NB), dtype=complex)  # Define y as a complex array
    y_shunt = np.zeros((NB, NB), dtype=complex)  # Define y_shunt as a complex array
    for i, row in net.line.iterrows():
        from_bus_idx = int(row['from_bus'])  # Convert to integer index
        to_bus_idx = int(row['to_bus'])      # Convert to integer index
        y[from_bus_idx, to_bus_idx] = -1/(row['length_km']*(row['r_ohm_per_km']+1j*row['x_ohm_per_km']))
        y_shunt[from_bus_idx, to_bus_idx] = 1j*row['c_nf_per_km']/2*1e-9*row['length_km']
        y[to_bus_idx, from_bus_idx] = -1/(row['length_km']*(row['r_ohm_per_km']+1j*row['x_ohm_per_km']))
        y_shunt[to_bus_idx, from_bus_idx] = 1j*row['c_nf_per_km']/2*1e-9*row['length_km']
    
    Y_bus = y
    for i in range(NB):
        y_diag = 0
        for k in range(NB):
            y_diag = y_diag-y[i, k] + y_shunt[i, k]
        Y_bus[i, i] = y_diag

    return Y_bus
    

In [ ]:

# import Flexibility
# from Flexibility import *
# importlib.reload(Flexibility)
import gurobipy as gp
import numpy as np
import pandas as pd

m = gp.Model('flex')

net_meas = net_meas

active_power_bus = {}
reactive_power_bus = {}
voltage_real_bus = {}
voltage_imag_bus = {}
flex_up = {}
flex_down = {}
flex_up_reactive = {}
flex_down_reactive = {}
node_number = len(net_meas.bus)
buses = [str(x) for x in np.arange(node_number)] 
NL = len(net.line)
lines = net.line

for i in range(len(buses)):
    # active_power_bus[i] = m.addVar(lb = 0,name = 'active_power_bus')
    # reactive_power_bus[i] = m.addVar(lb = -99.9,name = 'reactive_power_bus')
    voltage_real_bus[i] = m.addVar(lb = -99.9,name = 'voltage_real_bus')
    voltage_imag_bus[i] = m.addVar(lb = -99.9, name = 'voltage_imag_bus')
    flex_up[i] = m.addVar(name = 'flex_up')
    flex_down[i] = m.addVar(name = 'flex_down')
    flex_down_reactive[i] = m.addVar(name = 'flex_down_reactive')
    flex_up_reactive[i] = m.addVar(name = 'flex_up_reactive')
    

C = {}
S = {}


for i in buses:
    for j in buses:
        C[int(i), int(j)] = m.addVar(lb = -99.9 ,name = 'C')
        S[int(i), int(j)] = m.addVar (lb = -99.9  ,name='S')




# m.addConstrs((active_power_bus[bus] == 0 for bus in non_generate_ac_bus),name="balancePVP")
# m.addConstrs((reactive_power_bus[bus] == 0 for bus in non_generate_reac_bus),name="balancePVQ")

# m.addConstrs((active_power_bus[bus] <= Gupper[bus] for bus in generate_ac_bus),name="active_Capacity_upper")
# m.addConstrs((active_power_bus[bus >= Glower[dbus for busn generate_ac_dbusname="active_Capacity_lower")

# m.addConstrs((reactive_power_bus[bus] <= pvqmax[bus for dbusin generate_reac_bus),name="reactive_Capacity_upper")
# m.addConstrs((reactive_power_busbus >= pvqmin[bus] for busin generate_reac_bus),name="reactive_Capacity_lower")

# m.addConstrs((voltage_real_busbus] == pvv_busbus] for busin generate_reac_bus),name='PV')

# active_load_bus= net.load['p_mw']
# reactive_load_bus = net.load['q_mvar']

net_meas.res_gen = fill_gen(net_meas)
net_meas.load = fill_load(net_meas)
net_meas.res_sgen = fill_sgen(net_meas)
net_meas.ext_grid = fill_ext_grid(net_meas)

active_power_bus = net_meas.res_gen['p_mw']
active_power_bus_sgen = net_meas.res_sgen['p_mw']
reactive_power_bus = net_meas.res_gen['q_mvar']
reactive_power_bus_sgen = net_meas.res_sgen['q_mvar']
active_power_load_bus = net_meas.load['p_mw']
reactive_power_load_bus = net_meas.load['q_mvar']
active_power_ext_grid = net_meas.res_ext_grid['p_mw']
reactive_power_ext_grid = net_meas.res_ext_grid['q_mvar']

# active_load_bus = dict(zip(buses,powpdi.tolist()))
# reactive_load_bus = dict(zip(buses,powqdj.tolist()))

Y_bus = create_Y_bus(net_meas)

B = np.imag(Y_bus)
G = np.real(Y_bus)

def constrainp(i):
    a = B[i, i] * S[i, i]
    for j in buses:
        j = int(j)
        a += G[i,j]*C[i,j]-B[i,j]*S[i,j]
    return (a)

def constrainq(i):
    a = -G[i, i] * S[i, i]
    for j in buses:
        j = int(j)
        a += -B[i,j]*C[i,j]-G[i,j]*S[i,j]
    return (a)

def fintstr(bus):
    bus = int(bus)
    bus = str (bus)
    return bus

# def lineconstraints(i):
#     for j in buses:
#         (B[i, j]*B[i, j] + G[i, j])(*G[i, j]*C[i, i]-2*C[i, j]+C[j, j])




# m.addConstrs((constrainp(int(i))==active_power_bus[int(i)] - active_load_bus[int(i)]+flex_up[int(i)]-flex_down[int(i)]) for i in buses)
# m.addConstrs((constrainq(int(i))==reactive_power_bus[i] - reactive_load_bus[i]) for i in buses)
m.update()

m.addConstrs((constrainp(int(i))==active_power_bus[int(i)]-active_power_load_bus[int(i)]+active_power_bus_sgen[int(i)]+active_power_ext_grid[int(i)]+flex_up[int(i)]-flex_down[int(i)]) for i in buses)
# m.addConstrs((constrainp(int(i))==active_power_bus[int(i)]) for i in range(len(buses)))
m.addConstrs((constrainq(int(i))==reactive_power_bus[int(i)]-reactive_power_load_bus[int(i)]+reactive_power_bus_sgen[int(i)]+reactive_power_ext_grid[int(i)]+flex_up_reactive[int(i)]-flex_down_reactive[int(i)]) for i in range(len(buses)))
## Put reference bus
# m.addConstr(C[fintstr(data[2][1]),fintstr(data[2][1])]==1)'
m.addConstr(voltage_real_bus[0]==1)
m.addConstr(voltage_imag_bus[0] == 0)
m.addConstrs((C[bus_i, bus_i]<=1.05*1.05 for bus_i in range(len(buses))),name='ClimUP')
m.addConstrs((C[bus_i, bus_i]>=0.95*0.95 for bus_i in range(len(buses))),name='ClimLow')
m.addConstrs((C[bus_i, bus_j]==C[bus_j, bus_i] for bus_j in range(len(buses))  for bus_i in range(len(buses))),name='C_Constrs')
m.addConstrs((S[bus_i, bus_j]==-S[bus_j, bus_i] for bus_j in range(len(buses))  for bus_i in range(len(buses))),name='S_Constrs')



for bus_i in range(len(buses)):
    for bus_j in range(len(buses)):
        m.addQConstr((C[bus_i, bus_j]*C[bus_i, bus_j]+S[bus_i, bus_j]*S[bus_i, bus_j])<=C[bus_i, bus_i]*C[bus_j, bus_j])
c=100
obj = c*gp.quicksum((flex_up[n]-flex_down[n]+flex_up_reactive[n]-flex_down_reactive[n]) for n in range(len(buses)))

# obj = quicksum(Ga_bus[bus * active_power_bus[bus]* active_power_busbus] + Gb_busbus] * active_power_busbus]
#                          + Gc_bus[bus] for busin generate_ac_bus)

m.setObjective(obj, gp.GRB.MINIMIZE)
m.optimize()


In [ ]:
for i in range(len(net.bus)):
    print(f'flex_up_{i}', flex_up[i].X)
    print(f'flex_down_{i}', flex_down[i].X)
    print(f'flex_up_reactive{i}', flex_up_reactive[i].X)
    print(f'flex_down_reactive{i}', flex_down_reactive[i].X)
    print(f'voltage bus', voltage_real_bus[i].X, voltage_imag_bus[i].X)
print('total flex up:', np.sum(flex_up[i].X for i in range(len(net.bus))))
print('total flex down:', np.sum(flex_down[i].X for i in range(len(net.bus))))

In [ ]:
# Assuming `net` and the other variables (`flex_up`, `flex_down`, etc.) are already defined
flex_up_vals = []
flex_down_vals = []
flex_up_reactive_vals = []
flex_down_reactive_vals = []
voltage_real_vals = []
voltage_imag_vals = []

for i in range(len(net.bus)):
    flex_up_vals.append(flex_up[i].X)
    flex_down_vals.append(flex_down[i].X)
    flex_up_reactive_vals.append(flex_up_reactive[i].X)
    flex_down_reactive_vals.append(flex_down_reactive[i].X)
    voltage_real_vals.append(voltage_real_bus[i].X)
    voltage_imag_vals.append(voltage_imag_bus[i].X)

# Create a dictionary to store the data
flex_results = {
    'flex_up': flex_up_vals,
    'flex_down': flex_down_vals,
    'flex_up_reactive': flex_up_reactive_vals,
    'flex_down_reactive': flex_down_reactive_vals
    # 'voltage_real': voltage_real_vals,
    # 'voltage_imag': voltage_imag_vals
}

# Create the dataframe
flex_results_df = pd.DataFrame(flex_results)


In [ ]:
flex_results_df

In [ ]:
total_flex_available_up_active = 20
total_flex_available_up_reactive = 3
total_flex_available_down = 0
total_flex_available_down_reactive = 0


# import Flexibility
# from Flexibility import *
# importlib.reload(Flexibility)
import gurobipy as gp
import numpy as np
import pandas as pd

m = gp.Model('flex')

net_meas = net_meas

active_power_bus = {}
reactive_power_bus = {}
voltage_real_bus = {}
voltage_imag_bus = {}
flex_up = {}
flex_down = {}
flex_up_reactive = {}
flex_down_reactive = {}
node_number = len(net_meas.bus)
buses = [str(x) for x in np.arange(node_number)] 
NL = len(net.line)
lines = net.line

for i in range(len(buses)):
    # active_power_bus[i] = m.addVar(lb = 0,name = 'active_power_bus')
    # reactive_power_bus[i] = m.addVar(lb = -99.9,name = 'reactive_power_bus')
    voltage_real_bus[i] = m.addVar(lb = -99.9,name = 'voltage_real_bus')
    voltage_imag_bus[i] = m.addVar(lb = -99.9, name = 'voltage_imag_bus')
    flex_up[i] = m.addVar(name = 'flex_up')
    flex_down[i] = m.addVar(name = 'flex_down')
    flex_down_reactive[i] = m.addVar(name = 'flex_down_reactive')
    flex_up_reactive[i] = m.addVar(name = 'flex_up_reactive')
    

C = {}
S = {}


for i in buses:
    for j in buses:
        C[int(i), int(j)] = m.addVar(lb = -99.9 ,name = 'C')
        S[int(i), int(j)] = m.addVar (lb = -99.9  ,name='S')




# m.addConstrs((active_power_bus[bus] == 0 for bus in non_generate_ac_bus),name="balancePVP")
# m.addConstrs((reactive_power_bus[bus] == 0 for bus in non_generate_reac_bus),name="balancePVQ")

# m.addConstrs((active_power_bus[bus] <= Gupper[bus] for bus in generate_ac_bus),name="active_Capacity_upper")
# m.addConstrs((active_power_bus[bus >= Glower[dbus for busn generate_ac_dbusname="active_Capacity_lower")

# m.addConstrs((reactive_power_bus[bus] <= pvqmax[bus for dbusin generate_reac_bus),name="reactive_Capacity_upper")
# m.addConstrs((reactive_power_busbus >= pvqmin[bus] for busin generate_reac_bus),name="reactive_Capacity_lower")

# m.addConstrs((voltage_real_busbus] == pvv_busbus] for busin generate_reac_bus),name='PV')

# active_load_bus= net.load['p_mw']
# reactive_load_bus = net.load['q_mvar']

net_meas.res_gen = fill_gen(net_meas)
net_meas.load = fill_load(net_meas)
net_meas.res_sgen = fill_sgen(net_meas)
net_meas.ext_grid = fill_ext_grid(net_meas)

active_power_bus = net_meas.res_gen['p_mw']
active_power_bus_sgen = net_meas.res_sgen['p_mw']
reactive_power_bus = net_meas.res_gen['q_mvar']
reactive_power_bus_sgen = net_meas.res_sgen['q_mvar']
active_power_load_bus = net_meas.load['p_mw']
reactive_power_load_bus = net_meas.load['q_mvar']
active_power_ext_grid = net_meas.res_ext_grid['p_mw']
reactive_power_ext_grid = net_meas.res_ext_grid['q_mvar']

# active_load_bus = dict(zip(buses,powpdi.tolist()))
# reactive_load_bus = dict(zip(buses,powqdj.tolist()))

Y_bus = create_Y_bus(net_meas)

B = np.imag(Y_bus)
G = np.real(Y_bus)

def constrainp(i):
    a = B[i, i] * S[i, i]
    for j in buses:
        j = int(j)
        a += G[i,j]*C[i,j]-B[i,j]*S[i,j]
    return (a)

def constrainq(i):
    a = -G[i, i] * S[i, i]
    for j in buses:
        j = int(j)
        a += -B[i,j]*C[i,j]-G[i,j]*S[i,j]
    return (a)

def fintstr(bus):
    bus = int(bus)
    bus = str (bus)
    return bus

# def lineconstraints(i):
#     for j in buses:
#         (B[i, j]*B[i, j] + G[i, j])(*G[i, j]*C[i, i]-2*C[i, j]+C[j, j])



# m.addConstrs((constrainp(int(i))==active_power_bus[int(i)] - active_load_bus[int(i)]+flex_up[int(i)]-flex_down[int(i)]) for i in buses)
# m.addConstrs((constrainq(int(i))==reactive_power_bus[i] - reactive_load_bus[i]) for i in buses)
m.update()

m.addConstrs((constrainp(int(i))==active_power_bus[int(i)]-active_power_load_bus[int(i)]+active_power_bus_sgen[int(i)]+active_power_ext_grid[int(i)]+flex_up[int(i)]-flex_down[int(i)]) for i in buses)
# m.addConstrs((constrainp(int(i))==active_power_bus[int(i)]) for i in range(len(buses)))
m.addConstrs((constrainq(int(i))==reactive_power_bus[int(i)]-reactive_power_load_bus[int(i)]+reactive_power_bus_sgen[int(i)]+reactive_power_ext_grid[int(i)]+flex_up_reactive[int(i)]-flex_down_reactive[int(i)]) for i in range(len(buses)))
## Put reference bus
# m.addConstr(C[fintstr(data[2][1]),fintstr(data[2][1])]==1)'
m.addConstr(voltage_real_bus[0]==1)
m.addConstr(voltage_imag_bus[0] == 0)
m.addConstrs((C[bus_i, bus_i]<=1.05*1.05 for bus_i in range(len(buses))),name='ClimUP')
m.addConstrs((C[bus_i, bus_i]>=0.95*0.95 for bus_i in range(len(buses))),name='ClimLow')
m.addConstrs((C[bus_i, bus_j]==C[bus_j, bus_i] for bus_j in range(len(buses))  for bus_i in range(len(buses))),name='C_Constrs')
m.addConstrs((S[bus_i, bus_j]==-S[bus_j, bus_i] for bus_j in range(len(buses))  for bus_i in range(len(buses))),name='S_Constrs')



for bus_i in range(len(buses)):
    for bus_j in range(len(buses)):
        m.addQConstr((C[bus_i, bus_j]*C[bus_i, bus_j]+S[bus_i, bus_j]*S[bus_i, bus_j])<=C[bus_i, bus_i]*C[bus_j, bus_j])
c=100
obj = c*gp.quicksum((flex_up[n]-flex_down[n]+flex_up_reactive[n]-flex_down_reactive[n]) for n in range(len(buses)))

# obj = quicksum(Ga_bus[bus * active_power_bus[bus]* active_power_busbus] + Gb_busbus] * active_power_busbus]
#                          + Gc_bus[bus] for busin generate_ac_bus)

m.setObjective(obj, gp.GRB.MINIMIZE)
m.optimize()


Convertion to Pyomo

In [ ]:
m.write("model.lp")

In [ ]:
solver = SolverFactory('ipopt')
if solver.available():
    print("Ipopt solver is available.")
else:
    print("Ipopt solver is not available.")

In [ ]:
from pyomo.environ import *

# Create a Pyomo model
model = ConcreteModel()

# Load the MPS file
# model = read("model.mps")

# Specify the solver
solver = SolverFactory('ipopt')
                       #'C:/Users/emijo/AppData/Local/anaconda3/envs/digitaltwinv1')  # Or any other solver like 'cbc'

# Solve the model
results = solver.solve('model.lp')

# Display results
model.display()


In [ ]:
import pyomo.environ as pyo
import numpy as np

# Define the total flexibility available
total_flex_available_up_active = 20
total_flex_available_up_reactive = 3
total_flex_available_down = 0
total_flex_available_down_reactive = 0

# Data and parameters
net_meas = net_meas  # assumed to be provided
node_number = len(net_meas.bus)
buses = [str(x) for x in np.arange(node_number)]
# buses = list(range(node_number))
lines = net.line
NL = len(net.line)

# Create a Pyomo model
model = pyo.ConcreteModel()

# Define sets
model.buses = pyo.Set(initialize=buses)

# Define variables
model.voltage_real_bus = pyo.Var(model.buses, bounds=(-99.9, None))
model.voltage_imag_bus = pyo.Var(model.buses, bounds=(-99.9, None))
model.flex_up = pyo.Var(model.buses, bounds=(0, total_flex_available_up_active))
model.flex_down = pyo.Var(model.buses, bounds=(0, total_flex_available_down))
model.flex_up_reactive = pyo.Var(model.buses, bounds=(0, total_flex_available_up_reactive))
model.flex_down_reactive = pyo.Var(model.buses, bounds=(0, total_flex_available_down_reactive))

# Define C and S matrices for buses
C = {}
S = {}
# model.C = pyo.Var(model.buses, model.buses, within=pyo.Reals, bounds=(-99.9, None))
# model.S = pyo.Var(model.buses, model.buses, within=pyo.Reals, bounds=(-99.9, None))
# model.C = pyo.Var(model.buses, model.buses, bounds=(-99.9, None))
# model.S = pyo.Var(model.buses, model.buses, bounds=(-99.9, None))

# Data from measurements
active_power_bus = net_meas.res_gen['p_mw']
active_power_bus_sgen = net_meas.res_sgen['p_mw']
reactive_power_bus = net_meas.res_gen['q_mvar']
reactive_power_bus_sgen = net_meas.res_sgen['q_mvar']
active_power_load_bus = net_meas.load['p_mw']
reactive_power_load_bus = net_meas.load['q_mvar']
active_power_ext_grid = net_meas.res_ext_grid['p_mw']
reactive_power_ext_grid = net_meas.res_ext_grid['q_mvar']

# Create Y_bus from your net_meas
Y_bus = create_Y_bus(net_meas)
B = np.imag(Y_bus)
G = np.real(Y_bus)

# Define constraints
def active_power_balance_rule(model, i):
    i = int(i) 
    return (B[i, i] * model.S[i, i] +
            sum(G[i, j] * model.C[i, j] - B[i, j] * model.S[i, j] for j in buses) ==
            active_power_bus[i] - active_power_load_bus[i] + active_power_bus_sgen[i] +
            active_power_ext_grid[i] + model.flex_up[i] - model.flex_down[i])

model.active_power_balance = pyo.Constraint(model.buses, rule=active_power_balance_rule)

def reactive_power_balance_rule(model, i):
    i = int(i) 
    return (-G[i, i] * model.S[i, i] +
            sum(-B[i, j] * model.C[i, j] - G[i, j] * model.S[i, j] for j in buses) ==
            reactive_power_bus[i] - reactive_power_load_bus[i] + reactive_power_bus_sgen[i] +
            reactive_power_ext_grid[i] + model.flex_up_reactive[i] - model.flex_down_reactive[i])

model.reactive_power_balance = pyo.Constraint(model.buses, rule=reactive_power_balance_rule)

# Voltage magnitude constraints
model.voltage_real_bus_ref = pyo.Constraint(expr=model.voltage_real_bus[0] == 1)
model.voltage_imag_bus_ref = pyo.Constraint(expr=model.voltage_imag_bus[0] == 0)

def voltage_bounds_rule(model, i):
    return (pyo.inequality(0.95**2, model.C[i, i], 1.05**2))

model.voltage_bounds = pyo.Constraint(model.buses, rule=voltage_bounds_rule)

# C and S symmetry constraints
def C_symmetry_rule(model, i, j):
    return model.C[i, j] == model.C[j, i]

model.C_symmetry = pyo.Constraint(model.buses, model.buses, rule=C_symmetry_rule)

def S_symmetry_rule(model, i, j):
    return model.S[i, j] == -model.S[j, i]

model.S_symmetry = pyo.Constraint(model.buses, model.buses, rule=S_symmetry_rule)

# Quadratic constraints
def quadratic_constraint_rule(model, i, j):
    return model.C[i, j]**2 + model.S[i, j]**2 <= model.C[i, i] * model.C[j, j]

model.quadratic_constraint = pyo.Constraint(model.buses, model.buses, rule=quadratic_constraint_rule)

# Objective function: minimize the sum of flexibilities
def objective_rule(model):
    return 100 * sum((model.flex_up[i] - model.flex_down[i] + model.flex_up_reactive[i] - model.flex_down_reactive[i])
                     for i in model.buses)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Make sure IPOPT is installed
#C:\Users\emijo\AppData\Local\anaconda3\envs\multienergyhub
results = solver.solve(model, tee=True)

# Output the results
model.display()


In [ ]:
import pyomo.environ as pyo
import numpy as np

# Create a Pyomo model
model = pyo.ConcreteModel()

# Define sets for buses
model.buses = pyo.Set(initialize=range(len(buses)))

# Define decision variables (flexibility, voltage, etc.)
model.flex_up = pyo.Var(model.buses, within=pyo.Reals)
model.flex_down = pyo.Var(model.buses, within=pyo.Reals)

# Example decision variables
model.voltage_real_bus = pyo.Var(model.buses, bounds=(-99.9, None))
model.voltage_imag_bus = pyo.Var(model.buses, bounds=(-99.9, None))

# Now, C and S will be Python dictionaries, not Pyomo variables
C = {}
S = {}

# Define constraints and rules for active and reactive power balance
def active_power_balance_rule(model, i):
    # Example constraint using model variables and Python dictionaries
    # Populate C[i, j] and S[i, j] as needed
    C[i] = model.voltage_real_bus[i] ** 2  # Example: calculating and storing something in C
    S[i] = model.voltage_imag_bus[i] ** 2  # Example: calculating and storing something in S

    # Return the balance equation (can be adjusted for your real equation)
    return (B[i, i] * S[i] +
            sum(G[i, j] * C[i] - B[i, j] * S[i] for j in model.buses) ==
            active_power_bus[i] - active_power_load_bus[i] + 
            active_power_bus_sgen[i] + active_power_ext_grid[i] + 
            model.flex_up[i] - model.flex_down[i])

model.active_power_balance = pyo.Constraint(model.buses, rule=active_power_balance_rule)

# Similarly, reactive power balance rule:
def reactive_power_balance_rule(model, i):
    # Example: Storing intermediate results in C and S
    C[i] = model.voltage_real_bus[i] * 2  # Example calculation
    S[i] = model.voltage_imag_bus[i] * 2  # Example calculation
    
    return (G[i, i] * C[i] +
            sum(B[i, j] * S[i] - G[i, j] * C[i] for j in model.buses) ==
            reactive_power_bus[i] - reactive_power_load_bus[i] +
            reactive_power_bus_sgen[i] + reactive_power_ext_grid[i] +
            model.flex_up[i] - model.flex_down[i])

model.reactive_power_balance = pyo.Constraint(model.buses, rule=reactive_power_balance_rule)

# Define the objective (as in your original model)
model.obj = pyo.Objective(expr=sum(model.flex_up[n] - model.flex_down[n] for n in model.buses), sense=pyo.minimize)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Replace with the appropriate solver
solver.solve(model)

# After solving, you can extract values of C and S from model variables
C = {i: pyo.value(model.voltage_real_bus[i])**2 for i in model.buses}
S = {i: pyo.value(model.voltage_imag_bus[i])**2 for i in model.buses}


In [ ]:
from pyomo.environ import *
import numpy as np
import pandas as pd

# Define the Pyomo model
model = ConcreteModel()

# Data
total_flex_available_up_active = 20
total_flex_available_up_reactive = 3
total_flex_available_down = 0
total_flex_available_down_reactive = 0

net_meas = net_meas  # Assuming `net_meas` is a Pandas DataFrame or similar object

# Define sets
node_number = len(net_meas.bus)
buses = list(np.arange(node_number))
lines = list(net.line)

# Variables
model.voltage_real_bus = Var(buses, bounds=(-99.9, None))
model.voltage_imag_bus = Var(buses, bounds=(-99.9, None))
model.flex_up = Var(buses)
model.flex_down = Var(buses)
model.flex_up_reactive = Var(buses)
model.flex_down_reactive = Var(buses)

model.C = Var(buses, buses, bounds=(-99.9, None))
model.S = Var(buses, buses, bounds=(-99.9, None))

# Parameters (these would be populated with actual data)
# Assuming net_meas has a column `res_gen` for active and reactive power values
model.active_power_bus = Param(buses, initialize=dict(net_meas.res_gen['p_mw']))
model.reactive_power_bus = Param(buses, initialize=dict(net_meas.res_gen['q_mvar']))
model.active_power_load_bus = Param(buses, initialize=dict(net_meas.load['p_mw']))
model.reactive_power_load_bus = Param(buses, initialize=dict(net_meas.load['q_mvar']))
model.active_power_bus_sgen = Param(buses, initialize=dict(net_meas.res_sgen['p_mw']))
model.reactive_power_bus_sgen = Param(buses, initialize=dict(net_meas.res_sgen['q_mvar']))
model.active_power_ext_grid = Param(buses, initialize=dict(net_meas.res_ext_grid['p_mw']))
model.reactive_power_ext_grid = Param(buses, initialize=dict(net_meas.res_ext_grid['q_mvar']))

# Admittance matrix (Y_bus) would be created based on the network data
Y_bus = create_Y_bus(net_meas)  # Assuming create_Y_bus is defined elsewhere

B = np.imag(Y_bus)
G = np.real(Y_bus)

# Constraints
def constrainp_rule(model, i):
    return sum(G[i,j]*model.C[i,j] - B[i,j]*model.S[i,j] for j in buses) == \
        model.active_power_bus[i] - model.active_power_load_bus[i] + model.active_power_bus_sgen[i] + model.active_power_ext_grid[i] + model.flex_up[i] - model.flex_down[i]

def constrainq_rule(model, i):
    return sum(-B[i,j]*model.C[i,j] - G[i,j]*model.S[i,j] for j in buses) == \
        model.reactive_power_bus[i] - model.reactive_power_load_bus[i] + model.reactive_power_bus_sgen[i] + model.reactive_power_ext_grid[i] + model.flex_up_reactive[i] - model.flex_down_reactive[i]

model.constrainp = Constraint(buses, rule=constrainp_rule)
model.constrainq = Constraint(buses, rule=constrainq_rule)

# Voltage constraints
model.voltage_real_bus_ref = Constraint(expr=model.voltage_real_bus[0] == 1)
model.voltage_imag_bus_ref = Constraint(expr=model.voltage_imag_bus[0] == 0)

model.C_limit_up = Constraint(buses, rule=lambda model, i: model.C[i,i] <= 1.05*1.05)
model.C_limit_low = Constraint(buses, rule=lambda model, i: model.C[i,i] >= 0.95*0.95)

model.C_symmetric = Constraint(buses, buses, rule=lambda model, i, j: model.C[i,j] == model.C[j,i])
model.S_antisymmetric = Constraint(buses, buses, rule=lambda model, i, j: model.S[i,j] == -model.S[j,i])

# Quadratic constraint for the lines
def line_limit_rule(model, i, j):
    return model.C[i,j]**2 + model.S[i,j]**2 <= model.C[i,i] * model.C[j,j]

model.line_limits = Constraint(buses, buses, rule=line_limit_rule)

# Objective function
def objective_rule(model):
    return 100 * sum(model.flex_up[n] - model.flex_down[n] + model.flex_up_reactive[n] - model.flex_down_reactive[n] for n in buses)

model.objective = Objective(rule=objective_rule, sense=minimize)

# Solve
solver = SolverFactory('ipopt')
solver.solve(model)

# To access the values, you can do model.<variable>.value
